In [5]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX

In [6]:
df=pd.read_csv("Ankleshwar.csv")
df=df.iloc[:,1:]
df

,Date,pm25
0,04-02-2019 19:00,70.52
1,04-02-2019 20:00,61.39
2,04-02-2019 21:00,59.94
3,04-02-2019 22:00,76.44
4,04-02-2019 23:00,98.54
...,...,...
33531,02-12-2022 22:00,115.37
33532,02-12-2022 23:00,104.06
33533,03-12-2022 00:00,110.59
33534,03-12-2022 01:00,122.87


In [7]:
df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y %H:%M')
df.set_index('Date', inplace=True)

# Target series
ts = df['pm25']
# Use data without NaNs for model training
ts_valid = ts.dropna()

# Fit a basic SARIMA model (you may optimize the order parameters)
# Trend Elements
# There are three trend elements that require configuration.

# They are the same as the ARIMA model; specifically:

# p: Trend autoregression order.
# d: Trend difference order.
# q: Trend moving average order.
# Seasonal Elements
# There are four seasonal elements that are not part of ARIMA that must be configured; they are:

# P: Seasonal autoregressive order.
# D: Seasonal difference order.
# Q: Seasonal moving average order.
# m: The number of time steps for a single seasonal period.
model = SARIMAX(ts_valid, order=(1,1,1), seasonal_order=(0,1,1,24))  # assuming daily seasonality (24-hour cycle)
results = model.fit(disp=False)
# Create full predictions for the entire index range
ts_pred = results.predict(start=ts.index[0], end=ts.index[-1])

# Fill NaNs with predictions
ts_imputed = ts.copy()
ts_imputed[ts_imputed.isna()] = ts_pred[ts_imputed.isna()]
df['pm25_sarima'] = ts_imputed


C:\Users\Subham\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\Subham\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


MemoryError: Unable to allocate 560. MiB for an array with shape (51, 51, 28242) and data type float64

In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX

# # 🧹 Step 1: Load and prepare the data
# df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y %H:%M')
# df.set_index('Date', inplace=True)

# 🌫️ Step 2: Target series and clean data
ts = df['pm25']
ts_valid = ts.dropna()

# 🧽 Step 3: Optional downsampling to reduce memory load
# Use this only if hourly granularity isn't strictly needed
# ts_valid = ts_valid.resample('3H').mean().dropna()

# ⚙️ Step 4: Fit SARIMA with low_memory=True
model = SARIMAX(ts_valid, order=(1,1,1), seasonal_order=(1,1,1,24), enforce_stationarity=False, enforce_invertibility=False)
results = model.fit(disp=False, low_memory=True)

# 🔍 Step 5: Reindex to match original resolution for prediction
# Fill missing timestamps with NaNs at original frequency
ts_full = ts.copy()
ts_pred = results.predict(start=ts_full.index[0], end=ts_full.index[-1])

# 🧩 Step 6: Fill NaNs with predicted values
ts_imputed = ts_full.copy()
ts_imputed[ts_imputed.isna()] = ts_pred[ts_imputed.isna()]
df['pm25_sarima'] = ts_imputed

C:\Users\Subham\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\Subham\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


In [ ]:
df.to_csv("SArima.csv")